# Questão 2 
## Implementação do modelo com K-Nearest Neighbors

In [ ]:
import pandas as pd
from ucimlrepo import fetch_ucirepo
# Questão 2 A

# 1) Carregar a base de dados
cdc = fetch_ucirepo(id=891)
X_all = cdc.data.features
y = cdc.data.targets.squeeze().rename('Diabetes_binary')

# 2) Combinar features e target num DataFrame
df = pd.concat([X_all, y], axis=1)

# 3) Identificar variável-alvo
#    - “Diabetes_binary”: 0 = não diagnosticado, 1 = diagnosticado
print(df['Diabetes_binary'].value_counts())

# 4) Selecionar variáveis explicativas
#    Conforme analise previa em Q1
features = ['GenHlth', 'HighBP', 'HighChol', 'BMI', 'PhysActivity']
X = df[features]

# 5) Justificativa concisa
#    - GenHlth e HighBP foram as top correlacionadas com o target (|r| ≈ 0.30 e 0.20).  
#    - HighChol e BMI apresentam forte evidência clínica e correlação moderada.  
#    - PhysActivity capturou padrão não-linear relevante via informação mútua.  
#   Essas 5 variáveis formam um conjunto enxuto e informativo, otimizado para inferir risco de diabetes.


Diabetes_binary
0    218334
1     35346
Name: count, dtype: int64


In [6]:
from sklearn.model_selection import train_test_split
# Questão 2 B
# Supondo X e y já definidos:
# X: DataFrame com as features selecionadas
# y: Série com a variável-alvo 'Diabetes_binary'

# Realiza a separação em treino e teste de forma estratificada
X_train, X_test, y_train, y_test = train_test_split(
    X,                # Features
    y,                # Target
    test_size=0.2,    # 20% dos dados para teste
    random_state=42,  # garante reprodutibilidade
    stratify=y        # preserva a proporção de classes em ambos conjuntos
)

# Verificação das proporções originais e pós-split
print("Distribuição original:", y.value_counts(normalize=True).to_dict())
print("Distribuição treino:  ", y_train.value_counts(normalize=True).to_dict())
print("Distribuição teste:   ", y_test.value_counts(normalize=True).to_dict())


Distribuição original: {0: 0.8606669820245979, 1: 0.13933301797540207}
Distribuição treino:   {0: 0.8606659965310628, 1: 0.13933400346893723}
Distribuição teste:    {0: 0.8606709239987386, 1: 0.13932907600126143}


## Questão 2 B:  Separação Estratificada em Treino e Teste

Para garantir que o modelo seja treinado e avaliado em subconjuntos com a mesma proporção de casos de “diabetes” e “não-diabetes” do dataset original, adotamos a **estratificação** no momento do split.

**Como foi feito**  
- Reservamos 20 % dos dados para o conjunto de teste e 80 % para o treino.  
- Aplicamos estratificação com base na variável-alvo Diabetes_binary, mantendo a fração de casos positivos (~14 %) e negativos (~86 %) em ambos os subsets.

**Por que é importante**  
1. **Avaliação Confiável**  
   - Preserva o desequilíbrio real da população, evitando métricas enviesadas (por exemplo, um teste com poucos casos positivos poderia superestimar a acurácia).  
2. **Treino Representativo**  
   - Assegura que o modelo aprenda padrões de ambas as classes na mesma proporção em que ocorrem na população, melhorando a generalização.  
3. **Comparabilidade**  
   - Mantém consistência estatística entre treino e teste, permitindo comparações justas entre diferentes modelos e configurações.  


In [7]:
# Questão 2 C – Normalização das Variáveis Numéricas

from sklearn.preprocessing import StandardScaler
import pandas as pd

# Instanciar o scaler
scaler = StandardScaler()

# 1) Ajustar e transformar X_train
X_train_scaled = pd.DataFrame(
    scaler.fit_transform(X_train),
    columns=X_train.columns,
    index=X_train.index
)

# 2) Transformar X_test com os parâmetros calculados no treino
X_test_scaled = pd.DataFrame(
    scaler.transform(X_test),
    columns=X_test.columns,
    index=X_test.index
)

# 3) Verificação das estatísticas após normalização
stats = pd.concat([
    X_train_scaled.mean().rename('mean'),
    X_train_scaled.std().rename('std')
], axis=1).round(3)

stats


,mean,std
GenHlth,0.0,1.0
HighBP,-0.0,1.0
HighChol,-0.0,1.0
BMI,0.0,1.0
PhysActivity,0.0,1.0


## Questão 2 C: Normalização das Variáveis Numéricas

Antes de treinar o K-Nearest Neighbors, todas as features numéricas foram **normalizadas** para garantir que cada dimensão contribua de forma equilibrada na medida de similaridade entre observações.

**Como fizemos**  
- Calculamos média e desvio-padrão em cada coluna usando somente o conjunto de treino.  
- Subtraímos a média e dividimos pelo desvio-padrão (Standard Scaling), de modo que cada feature tenha média ≈ 0 e desvio ≈ 1.  
- Aplicamos essas mesmas estatísticas ao conjunto de teste (sem recalcular), evitando vazamento de informação.

**Por que é necessário para o KNN**  
1. **Igualar Amplitudes**  
   - O KNN usa, por padrão, distância Euclidiana. Sem normalização, variáveis com maior escala (por exemplo, IMC variando de 15 a 50) dominariam totalmente o cálculo, enquanto variáveis binárias ou de menor amplitude seriam irrelevantes.  
2. **Equilíbrio de Importância**  
   - Após a padronização, cada feature contribui proporcionalmente, permitindo que fatores clínicos, comportamentais e de autoavaliação de saúde (todas na mesma escala) influenciem corretamente a definição de “vizinhança”.  
3. **Melhor Convergência e Robustez**  
   - Modelos baseados em distância tendem a apresentar desempenho mais estável e interpretável quando as features estão centralizadas e escalonadas, reduzindo vieses e melhorando a sensibilidade da triagem.

> **Resumo:** a normalização padroniza a escala de todas as variáveis, assegurando que o KNN trate cada dimensão de forma justa e extraia o máximo de informação de cada indicador clínico e comportamental.  


In [ ]:
#Questão 2D Otimização do KNN

import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

# 2) Pipeline de pré-processamento + KNN
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier())
])

# 3) Grade de hiperparâmetros incluindo weighting
param_grid = {
    'knn__n_neighbors': list(range(1, 31, 2)),   # valores ímpares de 1 a 29
    'knn__weights': ['uniform', 'distance'],     # uniform vs distance weighted
    'knn__metric': ['minkowski', 'euclidean']    # distância padrão
}

# 4) GridSearchCV estratificado, otimizando F1 da classe positiva
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid = GridSearchCV(
    pipe,
    param_grid,
    scoring='f1',
    cv=cv,
    n_jobs=-1,
    verbose=1,
    refit=True
)

# 5) Ajuste no conjunto de treino
grid.fit(X_train, y_train)

# 6) Melhor combinação
best_params = grid.best_params_
best_score = grid.best_score_
print(f"Melhores parâmetros: {best_params}")
print(f"Melhor F1 CV (classe1): {best_score:.4f}\n")

# 7) Avaliação final no teste
y_pred = grid.predict(X_test)
print("Relatório de Classificação (Conjunto de Teste):")
print(classification_report(y_test, y_pred, digits=4))


Fitting 5 folds for each of 60 candidates, totalling 300 fits
Melhores parâmetros: {'knn__metric': 'minkowski', 'knn__n_neighbors': 1, 'knn__weights': 'uniform'}
Melhor F1 CV (classe1): 0.2733

Relatório de Classificação (Conjunto de Teste):
              precision    recall  f1-score   support

           0     0.8843    0.8929    0.8886     43667
           1     0.2962    0.2785    0.2871      7069

    accuracy                         0.8073     50736
   macro avg     0.5903    0.5857    0.5878     50736
weighted avg     0.8024    0.8073    0.8048     50736



# Comentários e Análise dos Resultados (Questão 2D)

## 1. Otimização via GridSearchCV  
- **Melhores hiperparâmetros** encontrados:  
  - `metric='minkowski'` (distância Euclidiana)  
  - `n_neighbors=1`  
  - `weights='uniform'`  
- **Critério de otimização:** F1‐score da classe positiva (diabetes) em 5‐fold CV, resultando em F1 médio ≈ 0.2733.

## 2. Desempenho no Conjunto de Teste

| Classe | Precision | Recall  | F1-score | Support |
|:------:|:---------:|:-------:|:--------:|:-------:|
| 0      | 0.8843    | 0.8929  | 0.8886   | 43 667  |
| 1      | 0.2962    | 0.2785  | 0.2871   | 7 069   |
| **Overall** |          |         |          |         |
| Accuracy | **0.8073** |         |          | 50 736  |
| Macro avg | 0.5903    | 0.5857  | 0.5878   |         |
| Weighted avg | 0.8024 | 0.8073  | 0.8048   |         |

### Interpretação das Métricas

1. **Classe 0 (Não-diabetes)**  
   - Precision e recall elevados (> 0.88) — o modelo acerta a grande maioria dos não-diabéticos e faz poucos falsos positivos para essa classe.  
   - F1 ≈ 0.89 reflete alta confiabilidade na predição “não-diabetes”.

2. **Classe 1 (Diabetes)**  
   - **Recall ≈ 0.28**: identifica apenas ~28 % dos verdadeiros portadores de diabetes → **alto risco de falsos negativos**.  
   - **Precision ≈ 0.30**: das previsões “diabetes”, apenas ~30 % estão corretas → **muitos falsos positivos**, gerando sobrecarga de triagem.  
   - F1 ≈ 0.29 sinaliza desempenho limitado na detecção de casos críticos.

## 3. Por que K = 1?

- **Maximiza o F1 médio em CV**, capturando padrões locais da classe minoritária.  
- **Contras de K = 1**:  
  - Tende a **overfitting**, sensível a ruídos e outliers — explicando recall e precision instáveis mesmo em CV.  
  - Alta variância entre folds (conforme boxplots anteriores), refletindo pouca robustez.

## 4. Consequências e Próximos Passos

1. **Trade-off Sensibilidade vs. Robustez**  
   - Embora K = 1 alcance o maior F1 em CV, seu baixo recall/teste e alta variância tornam-no pouco confiável em produção.  
2. **Experimentar `weights='distance'`**  
   - Atribuir peso maior a vizinhos mais próximos pode atualizar K = 1 para um comportamento mais suave, reduzindo falsos positivos sem sacrificar recall.  
3. **Avaliar Ks Médios (3–7)**  
   - Esses valores costumam manter F1 próximo ao máximo, com menor variabilidade e menos overfitting.  
4. **Ajustar Limiar de Decisão**  
   - Em vez de usar o limiar padrão de 0.5, alterar o threshold de probabilidade pode melhorar recall (captura mais positivos).  
5. **Comparar com Outros Modelos**  
   - Testar Regressão Logística, Random Forest e ensembles, avaliando AUC-ROC, curva PR e matriz de confusão, para determinar se outros algoritmos superam o KNN.  

> **Resumo:**  
> O KNN otimizado com K = 1 e `uniform` weights fornece um baseline sensível à classe minoritária, mas seu **desempenho modesto** em recall e F1 na validação final sugere a necessidade de ajustes de weighting, limiares ou a adoção de modelos mais robustos para uma triagem de diabetes eficaz.  


In [ ]:
# Teste e tentativas adicionais para otimização dos modelos

import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import classification_report, make_scorer, f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier


# 1) Configuração comum
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scorer = make_scorer(f1_score)

# 2) Pipelines com pré-processamento + modelo
pipelines = {
    'KNN': Pipeline([
        ('poly', PolynomialFeatures(degree=2, interaction_only=True)),
        ('scaler', StandardScaler()),
        ('clf', KNeighborsClassifier())
    ]),
    'Logistic': Pipeline([
        ('poly', PolynomialFeatures(degree=2, interaction_only=True)),
        ('scaler', StandardScaler()),
        ('clf', LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42))
    ]),
    'RandomForest': Pipeline([
        ('poly', PolynomialFeatures(degree=2, interaction_only=True)),
        ('scaler', StandardScaler()),
        ('clf', RandomForestClassifier(class_weight='balanced', random_state=42))
    ])
}

# 3) Grades de hiperparâmetros
param_grids = {
    'KNN': {
        'clf__n_neighbors': list(range(1, 16, 2)),
        'clf__weights': ['uniform','distance'],
        'clf__metric': ['minkowski','manhattan']
    },
    'Logistic': {
        'clf__C': [0.01, 0.1, 1, 10]
    },
    'RandomForest': {
        'clf__n_estimators': [100, 200],
        'clf__max_depth': [None, 5, 10],
        'clf__min_samples_leaf': [1, 5]
    }
}

# 4) GridSearchCV para cada modelo
best_results = {}
for name in pipelines:
    grid = GridSearchCV(
        pipelines[name],
        param_grids[name],
        scoring=scorer,
        cv=cv,
        n_jobs=-1
    )
    grid.fit(X_train, y_train)
    best_results[name] = {
        'estimator': grid.best_estimator_,
        'cv_f1': grid.best_score_,
        'params': grid.best_params_
    }

# 5) Avaliação final no teste
summary = []
for name, info in best_results.items():
    model = info['estimator']
    y_pred = model.predict(X_test)
    rpt = classification_report(y_test, y_pred, output_dict=True)
    summary.append({
        'Model': name,
        'CV F1': info['cv_f1'],
        'Test Precision (1)': rpt['1']['precision'],
        'Test Recall (1)': rpt['1']['recall'],
        'Test F1 (1)': rpt['1']['f1-score']
    })

results_df = pd.DataFrame(summary).set_index('Model')
results_df
